BI名企班，谢雅楠，20210314
### Thinking
1. 请简述基于蒙特卡洛的强化学习的原理？  
蒙特卡罗方法利用经验平均对策略值函数进行估计。当值函数被估计出来后，对于每个状态s ，通过最大化动作值函数，来进行策略的改进。蒙特卡洛搜索树是通过模拟来进行预测输出结果，理论上可以用于以{state,action}为定义的任何领域。  
Selection -- 选择，从根节点开始，按一定策略，搜索到叶子节点；  
Expansion -- 扩展，对叶子节点扩展一个或多个合法的子节点；  
Simulation -- 对子节点采用随机的方式（这也是为什么称之为蒙特卡洛的原因）模拟若干次实验。模拟到最终状态时即可得到当前模拟所得的分数；  
Backpropagation -- 回传，根据子节点若干次模拟的得分，更新当前子节点的模拟次数与得分值。同时将模拟次数与得分值回传到其所有祖先节点并更新祖先节点。    
在围棋中：每个节点代表一个局面，A/B代表被访问B次，黑棋赢了A次，我们将不断重复一个过程：  
Step1，选择Selection，从根节点往下走，每次都选一个“最有价值的子节点”，直到找到“存在未扩展的子节点”，即这个局面存在未走过的后续着法的节点，比如 3/3 节点  
Step2，扩展Expansion，给这个节点加上一个 0/0 子节点，对应之前所说的“未扩展的子节点”  
Step3，模拟Simluation，用快速走子策略（Rollout policy）走到底，得到一个胜负结果  
Step4，回传Backup，把模拟的结果加到它的所有父节点上，假设模拟的结果是 0/1，就把0/1加到所有父节点上  

2. 强化学习的目标与深度学习的目标有何区别？  
强化学习考虑的是个体（Agent）与环境（Environment）的交互问题，目标是找到一个最优策略，使Agent获得尽可能多的来自环境的奖励。  
深度学习的目标是减小特征输出值与（有监督性学习的）真实值之间的损失。



### Action（五子棋） 
棋盘大小 10 * 10  
采用强化学习（策略价值网络），用AI训练五子棋AI  
1）编写代码，说明神经网络（策略价值网络），MCTS原理  
2）生成五子棋AI模型 .model  
3）进行人机对弈

In [1]:
# 调用AI与人下五子棋
from __future__ import print_function
import pickle
from game import Board, Game #定义了棋盘Board
from mcts_pure import MCTSPlayer as MCTS_Pure
from mcts_alphaZero import MCTSPlayer
from policy_value_net_pytorch import PolicyValueNet  # Pytorch


# 由人来输入下棋的位置
class Human(object):
    def __init__(self):
        self.player = None

    def set_player_ind(self, p):
        self.player = p

    # 通过input交互，得到用户的下棋位置 move
    def get_action(self, board):
        try:
            location = input("输入你下棋的位置 x,y: ")
            print(location)
            if isinstance(location, str):  # for python3
                location = [int(n, 10) for n in location.split(",")]
            move = board.location_to_move(location)
        except Exception as e:
            move = -1
        if move == -1 or move not in board.availables:
            print("输入位置非法")
            move = self.get_action(board)
        return move

    def __str__(self):
        return "Human {}".format(self.player)


# GoBang主程序
def run():
    n = 5
    # 这里可以修改棋盘的大小，需要和AI Model的棋盘大小相等
    width, height = 10, 10
    # 调用AI模型
    model_file = 'best_policy.model'
    try:
        # 初始化棋盘
        board = Board(width=width, height=height, n_in_row=n)
        game = Game(board)

        # ############### human VS AI ###################
        # 加载AI Model
        best_policy = PolicyValueNet(width, height, model_file = model_file, use_gpu=False)
        # 设置n_playout越大，效果越好，不需要设置is_selfplay，因为不需要进行AI训练
        mcts_player = MCTSPlayer(best_policy.policy_value_fn, c_puct=5, n_playout=400)  

        # 也可以使用MCTS_Pure进行对弈，但是它太弱了
        # mcts_player = MCTS_Pure(c_puct=5, n_playout=1000)

        # 创建人类player, 输入下棋位置比如 3,3
        human = Human()

        # start_player=1表示电脑先手，0表示人先手
        game.start_play(human, mcts_player, start_player=1, is_shown=1)
    except KeyboardInterrupt:
        print('\n\rquit')


if __name__ == '__main__':
    run()


Player 1 with X
Player 2 with O

       0       1       2       3       4       5       6       7       8       9

   9   _       _       _       _       _       _       _       _       _       _    


   8   _       _       _       _       _       _       _       _       _       _    


   7   _       _       _       _       _       _       _       _       _       _    


   6   _       _       _       _       _       _       _       _       _       _    


   5   _       _       _       _       _       _       _       _       _       _    


   4   _       _       _       _       _       _       _       _       _       _    


   3   _       _       _       _       _       _       _       _       _       _    


   2   _       _       _       _       _       _       _       _       _       _    


   1   _       _       _       _       _       _       _       _       _       _    


   0   _       _       _       _       _       _       _       _       _       _    




/Users/xieyanan/Desktop/qqqqqq/policy_value_net_pytorch.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x_act = F.log_softmax(self.act_fc1(x_act))
/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Player 1 with X
Player 2 with O

       0       1       2       3       4       5       6       7       8       9

   9   _       _       _       _       _       _       _       _       _       _    


   8   _       _       _       _       _       _       _       _       _       _    


   7   _       _       _       _       _       _       _       _       _       _    


   6   _       _       _       _       _       _       _       _       _       _    


   5   _       _       _       _       O       _       _       _       _       _    


   4   _       _       _       _       _       _       _       _       _       _    


   3   _       _       _       _       _       _       _       _       _       _    


   2   _       _       _       _       _       _       _       _       _       _    


   1   _       _       _       _       _       _       _       _       _       _    


   0   _       _       _       _       _       _       _       _       _       _    


输入你下棋的位置 x,y: 5

输入你下棋的位置 x,y: 6,6
6,6
Player 1 with X
Player 2 with O

       0       1       2       3       4       5       6       7       8       9

   9   _       _       _       _       _       _       _       _       _       _    


   8   _       _       _       _       _       _       _       _       _       _    


   7   _       _       _       _       X       _       _       _       _       _    


   6   _       _       _       _       O       O       X       _       _       _    


   5   _       _       _       _       O       X       _       _       _       _    


   4   _       _       _       _       O       X       _       _       _       _    


   3   _       _       _       _       O       _       _       _       _       _    


   2   _       _       _       _       X       _       _       _       _       _    


   1   _       _       _       _       _       _       _       _       _       _    


   0   _       _       _       _       _       _       _       _       _       _

Player 1 with X
Player 2 with O

       0       1       2       3       4       5       6       7       8       9

   9   _       _       _       _       _       _       _       _       _       _    


   8   _       _       _       _       _       _       _       _       _       _    


   7   _       _       _       _       X       _       _       _       _       _    


   6   _       _       _       X       O       O       X       _       _       _    


   5   _       _       _       O       O       X       O       X       _       _    


   4   _       _       X       O       O       X       O       _       _       _    


   3   _       _       _       X       O       O       _       _       _       _    


   2   _       _       _       _       X       _       _       _       _       _    


   1   _       _       _       _       _       _       _       _       _       _    


   0   _       _       _       _       _       _       _       _       _       _    


输入你下棋的位置 x,y: 6